1KG
==
Run mushi on 3-SFS computed from 1000 Genome Project data

In [ ]:
%matplotlib inline 
# %matplotlib notebook
from mushi import kSFS
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA, NMF
from umap import UMAP
import pickle
import composition as cmp

In [ ]:
# plt.style.use('dark_background')

In [ ]:
pops = {}
with open('data/phase3_1000genomes/integrated_call_samples_v3.20130502.ALL.panel') as f:
    f.readline()
    for line in f:
        pop, super_pop = line.split('\t')[1:3]
        if super_pop not in pops:
            pops[super_pop] = []        
        if pop not in pops[super_pop]:
            pops[super_pop].append(pop)
pops['AFR'].append('Batwa')

generation time

In [ ]:
t_gen = 29

### Load 1KG AFR and Batwa 3-SFSs

In [ ]:
sorted_triplets = [f'{a5}{a}{a3}>{a5}{d}{a3}' for a in 'AC' for d in 'ACGT' if d != a for a5 in 'ACGT' for a3 in 'ACGT']

ksfs_dict = {}
for super_pop in pops:
    for pop in pops[super_pop]:
        ksfs_dict[pop] = pickle.load(open(f'1KG/scons_output/{pop}/mushi.pkl', 'rb'))

for pop in ksfs_dict:   
    foo, bar = ksfs_dict[pop].mutation_types.reindex(sorted_triplets)
    
    ksfs_dict[pop].mutation_types = foo
    ksfs_dict[pop].X = ksfs_dict[pop].X[:, bar]
    ksfs_dict[pop].μ.mutation_types = foo
    ksfs_dict[pop].μ.Z = ksfs_dict[pop].μ.Z[:, bar]    

In [ ]:
ksfs_dict['CEU'].μ.plot(t_gen=t_gen, clr=True, alpha=0.3, lw=3)
plt.xscale('log')
plt.xlim([None, 1e6])
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(20, 4.5))
for idx, super_pop in enumerate(('EUR',)):
    plt.sca(axes[idx])
    plt.title(super_pop)
    for idx2, pop in enumerate(('CEU',)):
        ksfs_dict[pop].plot('TCC>TTC', clr=True, rasterized=True, label=pop, lw=3, alpha=0.5, c=f'C{idx2}')
    plt.xscale('log')
    if idx:
        plt.ylabel(None)
    else:
        plt.ylabel('TCC>TTC CLR composition of variants')
    plt.legend()
plt.tight_layout()
plt.savefig(f'/Users/williamdewitt/Downloads/europulse.pdf')#, transparent=True)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(18, 4))
for idx, super_pop in enumerate(pops):
    plt.sca(axes[idx])
    plt.title(super_pop)
    for idx2, pop in enumerate(pops[super_pop]):
        ksfs_dict[pop].μ.plot(('TCC>TTC',), t_gen=t_gen, clr=True, label=pop, lw=3, alpha=0.5)
    plt.xscale('log')
#     plt.xlim([None, 1e6])
    if idx:
        plt.ylabel(None)
    else:
        plt.ylabel('TCC>TTC CLR composition of mutation intensity')
plt.tight_layout()
plt.savefig(f'/Users/williamdewitt/Downloads/europulse.mu.pdf')#, transparent=True)
plt.show()

In [ ]:
# for super_pop in pops:
#     print(super_pop)
#     plt.figure(figsize=(4, 4))
#     for pop in pops[super_pop]:
#         ksfs_dict[pop].η.plot(t_gen=t_gen, label=pop, lw=3, alpha=0.8, zorder=-1)
#     plt.xscale('log')
#     # plt.legend(loc=(1.04,0))
#     plt.tight_layout()
#     plt.savefig(f'/Users/williamdewitt/Downloads/{super_pop}.N.pdf', transparent=True)
#     plt.show()
    
    
fig, axes = plt.subplots(1, 5, sharey=True, figsize=(18, 4))
for idx, super_pop in enumerate(pops):
    plt.sca(axes[idx])
    plt.title(super_pop)
    for idx2, pop in enumerate(pops[super_pop]):
        ksfs_dict[pop].η.plot(t_gen=t_gen, label=pop, lw=3, alpha=0.5)
    plt.xscale('symlog')
#     plt.xlim([None, 1e6])
    if idx:
        plt.ylabel(None)
    else:
        plt.ylabel('$\eta(t)$')
plt.tight_layout()
plt.savefig(f'/Users/williamdewitt/Downloads/eta.pdf')#, transparent=True)
plt.show()

### Mutation type enrichment as a heatmap

k-SFS compositionally centralized across sample frequencies, and MUSH compositionally centralized across times

In [ ]:
center = 1 / len(sorted_triplets) # for vmin/vmax

for pop in ('Batwa',):#ksfs_dict:
    print(pop)
    
    singlets = ksfs_dict[pop].mutation_types.str[1].str.cat(ksfs_dict[pop].mutation_types.str[5], sep='>')
#     a5 = ksfs_dict[pop].mutation_types.str[0]
#     a3 = ksfs_dict[pop].mutation_types.str[2]

    col_map = {'A>C':'C0', 'A>G':'C1', 'A>T':'C2', 'C>A':'C3', 'C>G':'C4', 'C>T':'C5'}
    col_colors = [col_map[singlet] for singlet in singlets]
    
    g = ksfs_dict[pop].clustermap(figsize=(20, 7), col_cluster=False,
                                  xticklabels=True, rasterized=True,
                                  vmin=center / 1.7, vmax=1.7 * center,
                                  cmap='RdBu_r',
                                  col_colors=col_colors)
    g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 9, family='monospace')
#     g.ax_col_dendrogram.set_visible(False)
    g.savefig(f'/Users/williamdewitt/Downloads/heatmap.{pop}.pdf', transparent=True)
    plt.show()
    
    g = ksfs_dict[pop].μ.clustermap(t_gen=t_gen,
                                    figsize=(20, 7), col_cluster=False, xticklabels=True, rasterized=True,
                                    vmin=center / 1.5, vmax=1.5 * center,
                                    cmap='RdBu_r',
                                    col_colors=col_colors)
    g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 9, family='monospace')
#     g.ax_col_dendrogram.set_visible(False)
    g.savefig(f'/Users/williamdewitt/Downloads/heatmap.{pop}.mu.pdf', transparent=True)
    plt.show()    

### TMRCA CDF

In [ ]:
for super_pop in pops:
    print(super_pop)
    plt.figure(figsize=(3, 3))
    for pop in pops[super_pop]:
        plt.plot(ksfs_dict[pop].η.change_points, ksfs_dict[pop].tmrca_cdf(), label=pop)
        plt.xlabel('$t$')
        plt.ylabel('TMRCA CDF')
        plt.ylim([0, 1])
        plt.xscale('symlog')
        plt.legend()
        plt.tight_layout()
    plt.show()

Expectation of emprical spectra from counting variants, doing compositional data transformation, then pca

cite: https://link.springer.com/article/10.1023/A:1023818214614 and Aitchison

and see: https://stats.stackexchange.com/questions/305965/can-i-use-the-clr-centered-log-ratio-transformation-to-prepare-data-for-pca

In [ ]:
# PCA of integrated signal (expected seg sites)

import matplotlib.animation as animation
# from IPython.display import HTML

time = np.concatenate(([0], ksfs_dict[pop].μ.change_points))

# embedding = UMAP(n_components=2,
# #                  min_dist=0,
# #                  n_neighbors=10,
#                 )
embedding = PCA(n_components=2)#, whiten=True)
Z_dict = {}
for pop in ksfs_dict:
#     Z_dict[pop] = np.array([np.ravel(ilr(ksfs_dict[pop].μ.Z))])
    Z_dict[pop] = np.array([ilr((ksfs_dict[pop].L @ ksfs_dict[pop].μ.Z).sum(0, keepdims=True))])
#     Z_dict[pop] = np.array([ilr(ksfs_dict[pop].X.sum(0, keepdims=True))])
#     Z_dict[pop] = np.array([ilr(ksfs_dict[pop].μ.Z[0, :])])
#     Z_dict[pop] = np.array([ilr(ksfs_dict[pop].X[0, :])])

    
Z = np.concatenate(tuple(Z_dict[pop] for pop in ksfs_dict))
    
embedding.fit(Z)

# each
Z_transform_dict = {pop:embedding.transform(Z_dict[pop]) for pop in ksfs_dict}

fig, ax = plt.subplots(figsize=(5, 3))
for idx, super_pop in enumerate(pops):
    for pop in pops[super_pop]:
        plt.plot(*Z_transform_dict[pop][0, :],
                 'o',
                 markersize=5,
                 c=f'C{idx}',
                 label=pop)    
    #     ax.set_xlabel('UMAP 1')
    #     ax.set_ylabel('UMAP 2')
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')    
ax.legend(loc=(1.04, -0.15), fancybox=True, framealpha=0, ncol=2)
plt.tight_layout()
plt.savefig(f'/Users/williamdewitt/Downloads/X_pca.pdf', transparent=True)

In [ ]:
# sns.barplot(ksfs_dict[pop].mutation_types, embedding.components_[0,:])
# plt.show()

In [ ]:
Z_dict = {}
for pop in pops['AFR']:
    Z_dict[pop] = cmp.closure(ksfs_dict[pop].X.sum(0, keepdims=True))

df = pd.DataFrame(index=ksfs_dict[pop].μ.mutation_types)

df["5'"] = df.index.str[0]
df['mutation'] = df.index.str[1].str.cat(df.index.str[5], sep='→')
df["3'"] = df.index.str[2]

df['LWK Vs MSL'] = Z_dict['LWK'] / Z_dict['MSL']
df['YRI Vs MSL'] = Z_dict['YRI'] / Z_dict['MSL']
df['GWD Vs MSL'] = Z_dict['GWD'] / Z_dict['MSL']
df['ESN Vs MSL'] = Z_dict['ESN'] / Z_dict['MSL']

df = df.melt(id_vars=["5'", 'mutation', "3'"], var_name='comparison')

g = sns.FacetGrid(df, row='mutation', col='comparison',
                  row_order=('C→A', 'C→G', 'C→T', 'A→G', 'A→C', 'A→T'),
                  margin_titles=True,
                  height=1.5
                  )

def facet_heatmap(data, color, **kwargs):
    data = data.pivot(index="5'", columns="3'", values='value')
    sns.heatmap(data, **kwargs).invert_yaxis()

# colorbar axes
cbar_ax = g.fig.add_axes([1.1, .3, .05, .4])

g = g.map_dataframe(facet_heatmap,
                    cbar_ax=cbar_ax,
                    cmap='RdBu_r',
                    center=1,
                    vmin=0.84, vmax=1.16
                    )
for ax in g.axes.flat:
    plt.setp(ax.texts, text="") 
g.set_titles(row_template = '{row_name}', col_template = '{col_name}')

g.fig.tight_layout()
plt.tight_layout()
# so the colorbar doesn't overlap the plot
# g.fig.subplots_adjust(right=.9)
g.savefig('/Users/williamdewitt/Downloads/AFR_heatmap.pdf')
plt.show()

In [ ]:
time = np.concatenate(([0], ksfs_dict[pop].μ.change_points))[:-10]

Z_dict = {}
for pop in ksfs_dict:
    Z_dict[pop] = cmp.ilr(ksfs_dict[pop].μ.Z[:-10, :])
    
Z = np.concatenate(tuple(Z_dict[pop] for pop in ksfs_dict))

# use one of these, or the one above learned on the expected segregating sites
embedding = UMAP(n_components=2,
                 n_neighbors=20,
#                  min_dist=0,
# #                  metric='cosine',
                 local_connectivity=.01,
                 n_epochs=1000
                )
# embedding = PCA(n_components=2)#, whiten=True)

embedding.fit(Z)

# each
Z_transform_dict = {pop:embedding.transform(Z_dict[pop]) for pop in ksfs_dict}

plt.figure(figsize=(4, 4))
# plt.subplot(311)
for idx, super_pop in enumerate(pops):
    for idx2, pop in enumerate(pops[super_pop]):
        plt.plot(*Z_transform_dict[pop].T,
                 '-', lw=3, alpha=.75,
                 c=f'C{idx}',
                 label=super_pop if idx2 == 0 else None)
        plt.annotate(pop, Z_transform_dict[pop][0, :],
                     ha='center', va='center', c='w',
                     family='monospace' if pop is not 'Batwa' else None,
                     bbox=dict(boxstyle='circle', fc=f'C{idx}', ec=f'C{idx}', lw=2),
                     size=6)
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.tight_layout()
sns.despine()
plt.savefig(f'/Users/williamdewitt/Downloads/Z_embed.pdf')#, transparent=True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.gca(projection='3d')
ax.view_init(20, -30)
# plt.subplot(311)
for idx, super_pop in enumerate(pops):
    for idx2, pop in enumerate(pops[super_pop]):
        ax.plot(*Z_transform_dict[pop].T, np.log10(t_gen * time),
                '-', lw=3, alpha=.75,
                c=f'C{idx}',
                label=super_pop if idx2 == 0 else None)
        ax.scatter(*Z_transform_dict[pop][None, 0, :].T, np.log10(t_gen * time[1]), s=50,
                   c=f'C{idx}')
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
ax.set_zlabel('$\log_{10}(t)$')
# ax.zaxis._set_scale('log')
# ax.legend()
plt.tight_layout()
plt.savefig(f'/Users/williamdewitt/Downloads/Z_embed_3d.pdf')#, transparent=True)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(5,7))
for umap_comp in (0, 1):
    for idx, super_pop in enumerate(pops):
        for pop in pops[super_pop]:
            axes[umap_comp].plot(time, Z_transform_dict[pop][:, umap_comp],
                     '-', lw=3, alpha=.75,
                     c=f'C{idx}',
                     label=pop)
            axes[umap_comp].annotate(pop, (0, Z_transform_dict[pop][0, umap_comp]),
                         ha='center', va='center', c='w',
                         family='monospace' if pop is not 'Batwa' else None,
                         bbox=dict(boxstyle='circle', fc=f'C{idx}', ec=f'C{idx}', lw=2),
                         size=6)            
    axes[umap_comp].set_ylabel(f'UMAP {umap_comp + 1}')
plt.xlabel('time (generations)')
plt.xscale('symlog')
# plt.legend(loc=(1.04, -1), fancybox=True, framealpha=0)
plt.tight_layout()
plt.show()

In [ ]:
time = np.concatenate(([0], ksfs_dict[pop].μ.change_points))

embedding = NMF(alpha=1.5e0, n_components=10)#, verbose=False, tol=1e-10, max_iter=10000)
# embedding = PCA(n_components=3)
embedding.fit(np.concatenate(tuple(ksfs_dict[pop].μ.Z.T for pop in ksfs_dict)))

Z_transform_dict = {pop:embedding.transform(ksfs_dict[pop].μ.Z.T) for pop in ksfs_dict}

plt.figure(figsize=(4, 2))
for i in range(embedding.n_components):
    plt.plot(time, embedding.components_[i], label=f'latent history {i + 1}')
plt.xlabel('$t$')
plt.xscale('symlog')
plt.legend(loc='center left', prop={'size': 7.5}, framealpha=.5)
plt.show()

In [ ]:
for pop in ksfs_dict:
    print(pop)
    W = embedding.transform(ksfs_dict[pop].μ.Z.T).T

#     # norms = nmf.components_.T.mean(0, keepdims=True)
#     H = nmf.components_.T# / norms
#     # W = norms.T * W

    total_weight = W.sum(1, keepdims=True)
    W = W / total_weight

    df = pd.DataFrame(data=W.T,
                      index=ksfs_dict[pop].μ.mutation_types,
                      columns=range(1, n_components + 1))

    df_min = df.values.min()
    df_max = df.values.max()

    df["5'"] = df.index.str[0]
    df['mutation'] = df.index.str[1].str.cat(df.index.str[5], sep='→')
    df["3'"] = df.index.str[2]

    df = df.melt(id_vars=["5'", 'mutation', "3'"], var_name='latent history')

    g = sns.FacetGrid(df, row='mutation', col='latent history',
                      row_order=('C→A', 'C→G', 'C→T', 'A→G', 'A→C', 'A→T'),
                      margin_titles=True,
                      size=1.5
                      )

    def facet_heatmap(data, color, **kwargs):
        data = data.pivot(index="5'", columns="3'", values='value')
        sns.heatmap(data, **kwargs).invert_yaxis()n

    # colorbar axes
    cbar_ax = g.fig.add_axes([1.1, .3, .05, .4])

    g = g.map_dataframe(facet_heatmap,
                        cbar_ax=cbar_ax,
                        cmap='RdBu_r',
                        center=0,
                        vmin=df_min, vmax=df_max
                        )

    # so the colorbar doesn't overlap the plot
    # g.fig.subplots_adjust(right=.9)
#     plt.savefig('/Users/williamdewitt/Downloads/PC_heatmap.pdf')
    plt.show()

In [ ]:
plt.figure()
for pop in ksfs_dict:
    plt.scatter(*Z_transform_dict[pop][:, 1:].T, alpha=0.8)
plt.show()

In [ ]:
x = []
y = []
derived = []
context = []
pops = []

for i, triplet in enumerate(sorted_triplets):
    for pop in ksfs_dict:
        xy = Z_transform_dict[pop][i, 1:]
        x.append(xy[0])
        y.append(xy[1])
        pops.append(pop)
        context.append(triplet[:3])
        derived.append(triplet[5])
        
df = pd.DataFrame({'population':pops, 'context':context, 'derived':derived, 'latent history 2':x, 'latent history 3':y})

In [ ]:
g = sns.relplot(x='latent history 2', y='latent history 3', row='derived', col='context', data=df, hue='population',
           height=2, aspect=1, alpha=.8).set_titles("{col_name}>{row_name}")
g.savefig('/Users/williamdewitt/Downloads/AFR.signatures.pdf')
plt.show()

In [ ]:
sns.catplot(y='latent history 3', x='singlet', row='context', data=df, hue='population')
plt.show()